In [1]:
from PIL import Image
from io import BytesIO
from boto3 import client
import boto3
import time
import random
import json
import os
import pandas as pd
import random

from image_process_tools import *

import requests as req
import numpy as np
import gist
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import distance_metrics
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import metrics

from flask import (Flask,
                   jsonify,
                   render_template,
                   request)


In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
area = 'Boston-Massachusetts-US/'
bucket_name = "chen-gal-test"
imagepath = "https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbImages/"+area
imagepath

'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbImages/Boston-Massachusetts-US/'

In [4]:
prefix = "AirbnbImages/"+area
s3 = boto3.resource('s3')
bucket = s3.Bucket(name=bucket_name)
images_valid = list(bucket.objects.filter(Prefix=prefix))

In [16]:
filenames = []
apt_list = []
full_filenames = []
counter = 0
featureX = np.zeros((1,270))
count = 0

In [6]:
logfile_name = '../Logs/HSV_feature_extraction_Boston-Massachusetts.txt'
feature_folder = '../'+'features/'+ area + 'HSV/'

In [7]:
if not os.path.exists(feature_folder):
            os.makedirs(feature_folder)

In [7]:
combined_features = pd.concat(df_merged)

NameError: name 'df_merged' is not defined

In [23]:
for obj in images_valid[1:]:
    count += 1
    print("Processing image {} of {}\n ,{}..................".format(counter,len(images_valid),obj.key))
    string = obj.key.split('/')  
    
    try:
        full_path_name = 'https://s3-us-west-2.amazonaws.com/'\
                                +bucket_name+'/'+ str(obj.key)         
        response = req.get(full_path_name)
        img_temp = np.array(Image.open(BytesIO(response.content)))
#         im = np.array(img_temp.resize((600,400)))
        feature_tmp = hsv_hist_extract(img_temp,count)
    except:
        print('Error at img {}'.format(obj.key))
        with open(logfile_name,'a') as f:
            f.write('!!!!!!Failed at image {} ||{}\n'.format(counter,full_path_name) )       
        continue
    else:
        counter += 1
        full_filenames.append(full_path_name)
        apt_list.append(string[-1].split('_')[-2])
        filenames.append(string[-1])
        featureX = np.concatenate((featureX,feature_tmp.reshape(1,-1)),axis = 0)
        
        with open(logfile_name,'a') as f:
            f.write('---------------------\n \
                            successfully extracted the feature of image {} ||{}'.format(counter,full_path_name) )
       
    if counter % 5000 == 0:
        columns1 = list(range(featureX.shape[1]))
        df_return = pd.DataFrame(featureX[1:,:],columns = columns1)
        
        df_return['apt'] = apt_list
        df_return['full_filename'] = full_filenames
        df_return['filename'] = filenames
        
        ##

        
        if not os.path.exists(feature_folder):
            os.makedirs(feature_folder)        
        filename = feature_folder + 'featureture_'+str(counter)+'.pickle'
        df_return.to_pickle(filename)
        filenames,apt_list,full_filenames = [],[],[]
        featureX = np.zeros((1,270))
             
columns1 = list(range(featureX.shape[1]))
df_return = pd.DataFrame(featureX[1:,:],columns = columns1)

df_return['apt'] = apt_list
df_return['full_filename'] = full_filenames
df_return['filename'] = filenames
filename = feature_folder + 'featureure_'+str(counter)+'.pickle'
df_return.to_pickle(filename)

In [18]:
feature_folder

'../image/Boston-Massachusetts-US//new/'

In [19]:
pc = pd.read_pickle(filename)

In [20]:
pc[['full_filename','filename','apt']]

,full_filename,filename,apt
0,https://s3-us-west-2.amazonaws.com/chen-gal-te...,851927_5.jpg,851927
1,https://s3-us-west-2.amazonaws.com/chen-gal-te...,851927_6.jpg,851927
2,https://s3-us-west-2.amazonaws.com/chen-gal-te...,851927_7.jpg,851927
3,https://s3-us-west-2.amazonaws.com/chen-gal-te...,851927_8.jpg,851927
4,https://s3-us-west-2.amazonaws.com/chen-gal-te...,851927_9.jpg,851927
5,https://s3-us-west-2.amazonaws.com/chen-gal-te...,8552338_0.jpg,8552338
6,https://s3-us-west-2.amazonaws.com/chen-gal-te...,8552338_1.jpg,8552338
7,https://s3-us-west-2.amazonaws.com/chen-gal-te...,8552338_2.jpg,8552338
8,https://s3-us-west-2.amazonaws.com/chen-gal-te...,8552338_3.jpg,8552338
9,https://s3-us-west-2.amazonaws.com/chen-gal-te...,8552338_4.jpg,8552338
